In [64]:
from apps.patterns.models import OcurrenceReport
import numpy as np

LAST_COMMITS = 200
STUDY_APP = "orders"

reports = OcurrenceReport.objects.filter(app_name=STUDY_APP).order_by('-commited_epoch')[:LAST_COMMITS]

global_raw_data = {}
for rep in reports:
    report_acc = {}
    # Store the amount of hits per app per report
    for hit in rep.ocurrences:
        parts = hit.split("/")
        app_name = parts[1]
        if app_name in report_acc:
            element = report_acc[app_name]
            element += 1
            report_acc[app_name] = element
        else:
            report_acc[app_name] = 0
    
    # Order this shit by app name
    items = report_acc.items()
    report_acc = sorted(items)
    
    # Collect app count per report into a global datastore
    for app_name, app_count in report_acc:
        if app_name in global_raw_data:
            elements = global_raw_data[app_name]
            elements.append(app_count)
            global_raw_data[app_name] = elements
        else:
            global_raw_data[app_name] = [app_count]

In [65]:
import numpy as np

data = { app_name: np.array(global_raw_data[app_name]) for app_name in global_raw_data}

{ app_name: [np.average(data[app_name]), np.std(data[app_name])] for app_name in data }

{'adtrack': [2.0, 0.0],
 'api': [28.535714285714285, 0.7691434408828182],
 'assigner': [5.0, 0.0],
 'billing': [3.5142857142857142, 0.8576189154173505],
 'bizdev': [0.0, 0.0],
 'bookkeeping': [11.0, 0.0],
 'brandadmin_api': [1.0, 0.0],
 'brandcenter': [3.7, 0.458257569495584],
 'calls': [23.942857142857143, 0.23211538298959886],
 'capacity': [10.621428571428572, 0.7017107375433794],
 'capacity_automator': [0.0, 0.0],
 'carts': [1.0, 0.0],
 'catalog': [14.971428571428572, 0.16659862556700858],
 'catalog_stats': [3.0, 0.0],
 'catalog_workflow': [4.0, 0.0],
 'checkout': [23.0, 0.0],
 'city_management': [31.0, 0.0],
 'cornershop_payment_authorizers': [3.0, 0.0],
 'crm': [4.0, 0.0],
 'cs_adyen': [2.0, 0.0],
 'cs_stripe': [3.2785714285714285, 0.44829609384323443],
 'customer_incentives': [2.0, 0.0],
 'customer_orders_api': [1.0, 0.0],
 'customer_support': [9.0, 0.0],
 'dispatcher': [15.114285714285714, 1.6216645517391528],
 'estimator': [5.0, 0.0],
 'facturamx': [4.0, 0.0],
 'finance': [0.0,

## Generate ocurrences

* Generate a report:
  * every COMMIT_CHUNK commits (doing it for every commit is quite expensive) COMMIT_CHUNK=50
  * for POINTS_TO_ANALYZE times (POINTS_TO_ANALYZE=10)

In [5]:
COMMIT_CHUNK=50
POINT_TO_ANALYZE = 25
study_apps=dict(
    #catalog=dict(ignore_paths=["apps/catalog", "apps/promotions", "apps/stores"]),
    orders=dict(ignore_paths=["apps/orders"])
)

from apps.patterns.git import repo_at
from apps.patterns.models import OcurrenceReport
from apps.patterns.analyzer import analyze_repo_with_pattern
from apps.patterns.services import report_ocurrence
from apps.utils import to_arrow


commits_to_analyze = []

with repo_at(commit=None, force_delete=False, always_pull=True) as repo:
    points = POINT_TO_ANALYZE
    chunks_skipped = 0
    while points > 0:
        commit = list(repo.iter_commits("devel", max_count=COMMIT_CHUNK, skip=chunks_skipped))[0]
        commits_to_analyze.append(commit)
        print(f"Commit {commit.hexsha} at {to_arrow(commit.committed_datetime)}")
        points -= 1
        chunks_skipped += COMMIT_CHUNK
    

for app_name in study_apps:
    app_config = study_apps[app_name]
    for commit in commits_to_analyze:
        report_ocurrence(study_app=app_name, commit=commit.hexsha, verbose=True, **app_config)

Commit 43b9cdd7fd070dbaf029a655ebce96acd3d589df at 2020-11-03T17:21:19-03:00
Commit b6a2cbe6395326228535ab3127b62fddda4e29a6 at 2020-11-03T02:36:55-03:00
Commit 1bd9680e7ccbb640965ae0788491e0be49f668bb at 2020-11-02T14:48:22-03:00
Commit f2d2ab311c87839428bc79b1ce943190646af847 at 2020-11-02T12:22:57-03:00
Commit a2355340641dc356e0a4c757c9d7c335af3b01ab at 2020-10-30T20:58:48-03:00
Commit 6e7758abd22e4b348fddb2b3d445563c6300cbb2 at 2020-10-30T15:34:19-03:00
Commit 051e2e6619ed433f3effc6d943e979ab6815c115 at 2020-10-30T12:13:05-03:00
Commit 1d2e83d6419ed0e778a1657bcc4acf70a43acc3d at 2020-10-29T16:45:37-03:00
Commit e0af596d4c45e93597f4e7f00d5b3f6b743c84b2 at 2020-10-29T16:56:29-03:00
Commit 0e8f9d30673f788b949d911fb56f231e758c5d7e at 2020-10-29T10:42:02-03:00
Commit b5c4399bdf3ab79254aa932e02c8fa6dbff0d985 at 2020-10-28T19:02:59-03:00
Commit 49b384c110afecad10ca0818e674f8a6003a6cf0 at 2020-10-28T17:53:02-03:00
Commit 5a8d911eacaf5c59aa71baed48beb9272acd42bf at 2020-10-28T00:04:05-03:00

In [59]:
from apps.patterns.models import OcurrenceReport
OcurrenceReport.objects.count()

227